### wps_climdexInput_quantile

WPS wrapper for [climdexInput.quantile](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=15), which is a reimplementation of R’s type=8 created to improve the efficiency of this package.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url
from quail.utils import test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient("http://localhost:5005/wps")

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_quantile?

Signature:
quail.climdex_quantile(
    data_vector,
    quantiles_vector,
    loglevel='INFO',
    data_file=None,
    output_file='output.rda',
    vector_name='days',
)
Docstring:
Implements R’s type=8 in a more efficient manner

Parameters
----------
data_file : ComplexData:mimetype:`application/x-gzip`
    Path to the file containing data to compute quantiles on
data_vector : string
    R double vector data to compute quantiles on
quantiles_vector : string
    Quantiles to be computed
output_file : string
    Filename to store the output Rdata (extension .rda)
vector_name : string
    Name to label the output vector
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output_vector : string
    A vector of the quantiles in question
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/Desktop/quail/</home/sangwonl/Desktop/quail/venv/lib/python3.7/site-packages/birdy/client

### Run wps_climdex_quantile process

#### Run with data extracted from a file

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="quantile_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_quantile(
            data_file=resource_filename("tests","data/ec.1018935.rda"),
            data_vector="unlist(ec.1018935.tmax['MAX_TEMP'])",
            quantiles_vector="c(0.1, 0.5, 0.9)",
            vector_name="tmax_quantiles",
        )
quantile_url = output.get()[1]

In [7]:
test_rda_output(
    quantile_url, "tmax_quantiles", "expected_quantiles.rda", "expected_tmax_quantiles"
    )

#### Run with manually created data

In [8]:
with NamedTemporaryFile(suffix=".rda", prefix="quantile_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_quantile(
            data_vector="0:10",
            quantiles_vector="0.5",
            vector_name="tmax_quantile",
        )
quantile_data = output.get()[0]

In [9]:
assert quantile_data.split()[1] == "5"